In [2]:
#!/usr/bin/env python3

import mfem.ser as mfem
import numpy as np
import math
import matplotlib.pyplot as plt


def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """

    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()

    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    """
    # Define the target directory and file name
    target_dir = os.path.join(os.getcwd(), 'mesh', 'usr')
    file_name = f'{nx}x{ny}_2D.mesh'
    file_path = os.path.join(target_dir, file_name)
    
    # Check if the target directory exists; if not, create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Directory '{target_dir}' was created.")

    # Check if the file already exists
    if not os.path.exists(file_path):
        # If the file does not exist, write the mesh to the file
        mesh.Print(file_path)
        print(f"File '{file_path}' was successfully created.")
    else:
        print(f"File '{file_path}' already exists.")
    """
    return mesh


def assign_boundary_attributes(mesh, x_start, E_start, x_end, tol=1e-6):
    """
    Assigns boundary attributes to the mesh:
      - If any vertex of a boundary element is near (x_start, E_start), assign attribute 1.
      - Else if any vertex of a boundary element is near x_end, assign attribute 2.
      - Otherwise, leave the attribute as 0.
    
    Parameters:
      mesh    : The mfem.Mesh object.
      x_start : The x-coordinate for the left (inflow) boundary.
      E_start : The energy coordinate for the left boundary.
      x_end   : The x-coordinate for the right (outflow) boundary.
      tol     : Tolerance for comparing coordinates.
    """
    vertex_array = mesh.GetVertexArray()
    
    # Initialize all boundary element attributes to 0.
    for i in range(mesh.GetNBE()):
        mesh.SetBdrAttribute(i, 0)
    
    # Loop over each boundary element and assign attributes.
    for i in range(mesh.GetNBE()):
        v_indices = mesh.GetBdrElementVertices(i)
        for idx in v_indices:
            x_coord = vertex_array[idx][0]
            E_coord = vertex_array[idx][1]
            if abs(x_coord - x_start) < tol and abs(E_coord - E_start) < tol:
                mesh.SetBdrAttribute(i, 1)
                break
            elif abs(x_coord - x_end) < tol:
                mesh.SetBdrAttribute(i, 2)
                break


def gauss_legendre_dirs(N):
    """
    Returns discrete ordinates (mu) and weights (w) from Gauss-Legendre
    on [-1,1] with N points total.

    Args:
        N (int): number of discrete angles

    Returns:
        (mu_vals, w_vals): arrays of length N
    """
    mu_vals = np.zeros(N)
    w_vals = np.zeros(N)
    # We can use e.g. numpy.polynomial.legendre.leggauss:
    mu_n, w_n = np.polynomial.legendre.leggauss(N)
    for i in range(N):
        mu_vals[i] = mu_n[i]
        w_vals[i] = w_n[i]
    return mu_vals, w_vals


# ---------------------------------------------------
# Example placeholder coefficients used in your code
# ---------------------------------------------------

class TotalXSCoefficient(mfem.PyCoefficient):
    """A constant total cross section: returns self.val."""
    def __init__(self, val):
        super().__init__()
        self.val = val
    def EvalValue(self, ip):
        return self.val

class ScatteringXSCoefficient(mfem.PyCoefficient):
    """A constant scattering cross section."""
    def __init__(self, val):
        super().__init__()
        self.val = val
    def EvalValue(self, ip):
        return self.val

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """Toy stopping power S(E). We'll just return a constant for demonstration."""
    def __init__(self, val):
        super().__init__()
        self.val = val
    def EvalValue(self, ip):
        return self.val

class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """dS/dE as function of E. We'll do a piecewise or tabulated approach."""
    def __init__(self, dS_dE_arr, E_start, E_end):
        super().__init__()
        self.dS_dE_arr = dS_dE_arr
        self.E_start = E_start
        self.E_end = E_end
        # We'll assume dS_dE_arr is nE+1 in length, spanning [E_start -> E_end].
        # We can interpolate linearly.

    def EvalValue(self, ip):
        E = ip[1]
        # If E > E_start or E < E_end, clamp
        if E > self.E_start: 
            return self.dS_dE_arr[0]
        if E < self.E_end:
            return self.dS_dE_arr[-1]
        # Linear interpolation. 
        N = len(self.dS_dE_arr)-1
        dE = (self.E_start - self.E_end) / N
        idx = int((self.E_start - E)/dE)
        if idx < 0:
            idx = 0
        elif idx >= N:
            idx = N-1
        return self.dS_dE_arr[idx]

def compute_S_derivative(E_arr, S_arr):
    """
    Numerically compute dS/dE from tabulated S(E).
    E_arr, S_arr: arrays of same length
    Returns an array dS_dE of same length
    """
    N = len(E_arr)
    dS_dE = np.zeros(N)
    for i in range(N-1):
        dS = S_arr[i+1] - S_arr[i]
        dE = E_arr[i+1] - E_arr[i]
        dS_dE[i] = dS/dE if abs(dE)>1e-14 else 0.0
    dS_dE[-1] = dS_dE[-2]  # just replicate
    return dS_dE

class InflowCoefficient(mfem.PyCoefficient):
    """A simple inflow coefficient that returns a constant value."""
    def __init__(self, val):
        super().__init__()
        self.val = val
    def EvalValue(self, ip):
        return self.val

# A constantCoefficient helper
def constant(val):
    return mfem.ConstantCoefficient(val)

# -----------
# VelocityCoefficient:
# -----------
class VelocityCoefficient(mfem.PyCoefficient):
    """
    v(x,E) = mu * ...
    For demonstration, we'll just return mu. The snippet suggests
    we might incorporate S(E), but let's keep it simple.
    """
    def __init__(self, mu, S_arr, E_start, E_end):
        super().__init__()
        self.mu = mu
        # ignoring S_arr for the sake of demonstration
    def EvalValue(self, ip):
        # ip => (x, E)
        return self.mu


# -----------
# get_marker_for_mu
# -----------
def get_marker_for_mu(mesh, mu):
    """
    Creates an intArray that marks which boundary attributes
    are 'active' for inflow. For a standard left-boundary inflow,
    attribute=1 => inflow if mu>0
    If you want to handle right boundary inflow for mu<0, you'd
    set marker[1] = 1. (Because attribute=2 is x=L side.)

    So we do:
      marker[0] = 1 if mu>0 else 0   (this is for boundary attribute=1)
      marker[1] = 1 if mu<0 else 0   (this is for boundary attribute=2)
    But in your snippet you only do left inflow. We'll keep it simple.
    """
    bmax = mesh.bdr_attributes.Max()
    marker = mfem.intArray(bmax)
    marker.Assign(0)
    # attribute=1 is index=0
    if mu > 0:
        marker[0] = 1  # left side inflow
    # if you wanted right inflow for mu<0:
    # if mu < 0:
    #     marker[1] = 1
    return marker


# --------------------------
# Main code (like your snippet)
# --------------------------

if __name__ == "__main__":
    # Set parameters
    nx = 10
    nE = 30
    x_start = 0.0
    x_end   = 0.3
    E_start = 1.0
    E_end   = 0.01
    N_ang   = 4
    order   = 1

    # 1) Create the 2D (x,E) mesh
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
    dim = mesh.Dimension()

    # 2) Assign boundary attributes
    assign_boundary_attributes(mesh, x_start, E_start, x_end, tol=1e-6)

    # 3) Define a DG space
    fec = mfem.DG_FECollection(order, dim)
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetTrueVSize())

    # 4) Mark essential boundaries for left=1, right=2 if you want Dirichlet
    #    You did something like:
    ess_bdr = mfem.intArray(mesh.bdr_attributes.Max())
    ess_bdr.Assign(0)
    ess_bdr[0] = 1  # attribute=1 => left boundary
    ess_bdr[1] = 1  # attribute=2 => right boundary (if you want Dirichlet=0)

    # 5) Discrete angles
    mu_vals, w_vals = gauss_legendre_dirs(N_ang)

    # 6) Example data for cross sections and stopping power
    S_arr = np.linspace(0, 2, nE + 1)
    E_arr = np.linspace(E_start, E_end, nE + 1)
    xs_t_coeff  = TotalXSCoefficient(5)
    xs_s_coeff  = ScatteringXSCoefficient(5)
    S_coeff     = StoppingPowerCoefficient(1)
    dS_dE_arr   = -compute_S_derivative(E_arr, S_arr)
    dS_dE_coeff = StoppingPowerDerivativeCoefficient(dS_dE_arr, E_start, E_end)
    q_coeff     = mfem.ConstantCoefficient(100)
    inflow_coeff= InflowCoefficient(1.0)

    # 7) Project an initial condition or inflow
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)
    # Dirichlet BC projection on left boundary
    psi.ProjectBdrCoefficient(inflow_coeff, ess_bdr)

    psi_mu_list = []

    # 8) Loop over angles
    for mu, w in zip(mu_vals, w_vals):
        print("  Solving for mu =", mu)

        # Build velocity coefficient
        v_coeff = VelocityCoefficient(mu, S_arr, E_start, E_end)

        # Build the bilinear form
        a = mfem.BilinearForm(fes)
        # Example PDE parts:
        #   ∇·(v ψ) => ConvectionIntegrator
        a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff))
        #   Σ_t * ψ => "mass" integrator with coefficient
        a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
        #   dS/dE => also mass integrator if you treat it like Σ_t * ψ,
        #            but it's not exactly that. We'll follow your snippet:
        a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))

        # DG face terms: interior and boundary
        #   Typically for convection, you add a DGTraceIntegrator with upwind param
        a.AddInteriorFaceIntegrator(
            mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
        a.AddBdrFaceIntegrator(
            mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))

        a.Assemble()
        a.Finalize()
        A = a.SpMat()

        # Build the RHS
        b = mfem.LinearForm(fes)
        b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))

        # Mark which boundary gets inflow
        marker = get_marker_for_mu(mesh, mu)
        # Add boundary integrator for inflow
        b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -1.0), marker)

        b.Assemble()

        # Solve
        psi_local = mfem.GridFunction(fes)
        psi_local.Assign(1.0)

        prec = mfem.GSSmoother(A)
        solver = mfem.GMRESSolver()
        solver.SetOperator(A)
        solver.SetPreconditioner(prec)
        solver.SetRelTol(1e-12)
        solver.SetAbsTol(1e-12)
        solver.SetMaxIter(500)
        solver.SetKDim(30)
        solver.SetPrintLevel(0)
        solver.Mult(b, psi_local)

        print("    GMRES (mu =", mu, "): iterations =", solver.GetNumIterations(),
              "final norm =", solver.GetFinalNorm())

        # Store for post-processing
        psi_mu_list.append((mu, w, psi_local))

        # Optional: save each angular solution
        psi_local.Save("psi_mu_{:.3f}.gf".format(mu))

    # 9) Combine angular solutions -> scalar flux phi_new
    phi_new = mfem.GridFunction(fes)
    phi_new.Assign(0.0)
    for (mu, w, psi_loc) in psi_mu_list:
        phi_new.Add(w, psi_loc)
    phi_new.Save("phi_new.gf")

    # 10) Compute cell-average for plotting
    phi_arr = np.array([phi_new[i] for i in range(phi_new.Size())])

    num_cells = mesh.GetNE()
    cell_avg_2d = np.zeros((nE, nx))

    cell_counter = 0
    for ix in range(nx):
        for iE in range(nE):
            dof_indices = fes.GetElementVDofs(cell_counter)
            vals = [phi_new[d] for d in dof_indices]
            cell_avg_2d[iE, ix] = np.mean(vals)
            cell_counter += 1

    # Construct cell centers for x and E
    x_nodes = np.linspace(x_start, x_end, nx+1)
    cell_x = 0.5*(x_nodes[:-1] + x_nodes[1:])
    E_nodes = np.linspace(E_start, E_end, nE+1)
    cell_E = 0.5*(E_nodes[:-1] + E_nodes[1:])
    # Because E decreases from E_start to E_end, let's flip for plotting top-down
    cell_E = cell_E[::-1]
    # Also flip the data array
    cell_avg_2d = cell_avg_2d[::-1, :]

    # Make a 2D meshgrid for plotting
    X, E_grid = np.meshgrid(cell_x, cell_E)

    plt.figure(figsize=(8,6))
    cp = plt.contourf(X, E_grid, cell_avg_2d, levels=50, cmap='jet')
    plt.colorbar(cp)
    plt.xlabel("x (cm)")
    plt.ylabel("Energy (MeV)")
    plt.title("Cell-Averaged Scalar Flux Φ(x,E)")
    plt.show()
    print("Done. Plot displayed.")


: 

In [ ]:
print("mesh.GetNE() =", mesh.GetNE())


NameError: name 'mesh' is not defined